# DSAI-HW3

## Import package

In [32]:
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, TimeDistributed, Dense, RepeatVector, Activation, BatchNormalization,Bidirectional
from keras.models import load_model
import numpy as np
from six.moves import range

## Parameters Config

In [33]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [34]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS+6
chars = '0123456789-+xyzabc '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

## Data Generation

In [35]:
%%time
data = []
label = []
seen = set()

print('Generating data...')
while len(data) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    if(a<b):
        a,b = b,a
    
    operator = np.random.choice(list('+-'))
    
#     q = '{}-{}'.format(a, b)
    q = str(a) + operator + str(b)    
    
    if(operator == "-"):
        for i in range(len(str(b))):
            if(int(str(a)[-i-1]) - int(str(b)[-i-1]) < 0 ):
                if(i==0):
                    q+= "z"
                elif(i == 1):
                    q+= "y"
                else:
                    q+= "x"
                    
    elif(operator == '+'):
        for i in range(len(str(b))):
            if(int(str(a)[-i-1]) + int(str(b)[-i-1]) > 10 ):
                if(i==0):
                    q+= "c"
                elif(i == 1):
                    q+= "b"
                else:
                    q+= "a"


    if(q not in seen):
        query = q + ' ' * (MAXLEN - len(q))
        seen.add(query)
        data.append(query)
        
        if(operator == "+"):
            ans = str(a+b)
        else:
            ans = str(a-b)
        
        ans += ' '* (DIGITS + 1 - len(ans))
        label.append(ans)
        
    
# print(data)
# print(label)

Generating data...
Wall time: 5.54 s


## Processing

In [36]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [37]:
ctable = CharacterTable(chars)

print('Vectorization...')
x = np.zeros((len(data), MAXLEN, len(chars)))
y = np.zeros((len(label), DIGITS + 1, len(chars)))
for i, sentence in enumerate(data):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(label):
    y[i] = ctable.encode(sentence, DIGITS + 1)
    
# print(x.shape)
# print(y.shape)
# print(y)

Vectorization...


In [38]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:40000]
train_y = y[:40000]
test_x = x[40000:]
test_y = y[40000:]

split_at = len(train_x) - len(train_x) // 10
print(len(train_x))

(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

40000
Training Data:
(36000, 13, 19)
(36000, 4, 19)
Validation Data:
(4000, 13, 19)
(4000, 4, 19)
Testing Data:
(40000, 13, 19)
(40000, 4, 19)


## Build Model

In [39]:
print('Build model...')
model = Sequential()

model.add(BatchNormalization(input_shape =(MAXLEN,len(chars))))
model.add(Bidirectional(RNN(HIDDEN_SIZE),merge_mode='concat'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(layers.RepeatVector(DIGITS + 1))
model.add(RNN(HIDDEN_SIZE, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_7 (Batch (None, 13, 19)            76        
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               151552    
_________________________________________________________________
batch_normalization_8 (Batch (None, 256)               1024      
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 4, 256)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 4, 128)            197120    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4, 19)             2451  

## Training

In [40]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)



--------------------------------------------------
Iteration 0
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 28s 769us/step - loss: 1.2143 - acc: 0.5922 - val_loss: 0.8995 - val_acc: 0.6856
Q 766+46c       T 812  ☒ 782 
Q 447+24c       T 471  ☒ 451 
Q 33+30         T 63   ☒ 37  
Q 928-5         T 923  ☑ 923 
Q 699-2         T 697  ☒ 699 
Q 788+3c        T 791  ☒ 781 
Q 715+95        T 810  ☒ 700 
Q 462-1         T 461  ☒ 466 
Q 383-1         T 382  ☒ 336 
Q 94+6          T 100  ☒ 99  

--------------------------------------------------
Iteration 1
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 18s 495us/step - loss: 0.6037 - acc: 0.7989 - val_loss: 0.4174 - val_acc: 0.8613
Q 69+51b        T 120  ☑ 120 
Q 330+17        T 347  ☑ 347 
Q 87-15         T 72   ☑ 72  
Q 13-4z         T 9    ☑ 9   
Q 8+6c          T 14   ☑ 14  
Q 153-15z       T 138  ☒ 148 
Q 646-6         T 

36000/36000 [==============================] - 18s 494us/step - loss: 0.0318 - acc: 0.9901 - val_loss: 0.0457 - val_acc: 0.9838
Q 263+3         T 266  ☑ 266 
Q 566-254       T 312  ☒ 322 
Q 388+4c        T 392  ☑ 392 
Q 3-1           T 2    ☑ 2   
Q 94-38z        T 56   ☑ 56  
Q 263-7z        T 256  ☑ 256 
Q 273+92b       T 365  ☑ 365 
Q 174+6         T 180  ☑ 180 
Q 786+18c       T 804  ☑ 804 
Q 838+114c      T 952  ☑ 952 

--------------------------------------------------
Iteration 14
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 18s 490us/step - loss: 0.0238 - acc: 0.9930 - val_loss: 0.0416 - val_acc: 0.9848
Q 8+1           T 9    ☑ 9   
Q 40+39         T 79   ☑ 79  
Q 48-3          T 45   ☑ 45  
Q 5+4           T 9    ☑ 9   
Q 8+1           T 9    ☑ 9   
Q 614+483       T 1097 ☒ 1007
Q 44-7z         T 37   ☑ 37  
Q 37+9c         T 46   ☑ 46  
Q 689+502ca     T 1191 ☒ 1291
Q 34-4          T 30   ☑ 30  

-------------------

36000/36000 [==============================] - 17s 479us/step - loss: 0.0053 - acc: 0.9989 - val_loss: 0.0340 - val_acc: 0.9886
Q 33-3          T 30   ☑ 30  
Q 525-67zy      T 458  ☑ 458 
Q 8+1           T 9    ☑ 9   
Q 51+8          T 59   ☑ 59  
Q 41+0          T 41   ☑ 41  
Q 995+4         T 999  ☑ 999 
Q 950+0         T 950  ☑ 950 
Q 195-71        T 124  ☑ 124 
Q 74-14         T 60   ☑ 60  
Q 951+720a      T 1671 ☑ 1671

--------------------------------------------------
Iteration 27
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 18s 497us/step - loss: 0.0061 - acc: 0.9984 - val_loss: 0.0376 - val_acc: 0.9879
Q 777+7c        T 784  ☑ 784 
Q 650+1         T 651  ☑ 651 
Q 84-7z         T 77   ☑ 77  
Q 71+6          T 77   ☑ 77  
Q 28+0          T 28   ☑ 28  
Q 78-77         T 1    ☑ 1   
Q 47+6c         T 53   ☑ 53  
Q 8-4           T 4    ☑ 4   
Q 80+36b        T 116  ☑ 116 
Q 66+3          T 69   ☑ 69  

-------------------

36000/36000 [==============================] - 18s 489us/step - loss: 0.0056 - acc: 0.9984 - val_loss: 0.0323 - val_acc: 0.9911
Q 734+70        T 804  ☑ 804 
Q 676+18c       T 694  ☑ 694 
Q 949-53y       T 896  ☑ 896 
Q 18+7c         T 25   ☑ 25  
Q 1-1           T 0    ☑ 0   
Q 389-0         T 389  ☑ 389 
Q 44-8z         T 36   ☑ 36  
Q 97-1          T 96   ☑ 96  
Q 49-2          T 47   ☑ 47  
Q 76+1          T 77   ☑ 77  

--------------------------------------------------
Iteration 40
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 18s 488us/step - loss: 0.0068 - acc: 0.9980 - val_loss: 0.0325 - val_acc: 0.9900
Q 97+2          T 99   ☑ 99  
Q 5-2           T 3    ☑ 3   
Q 23-1          T 22   ☑ 22  
Q 36-35         T 1    ☑ 1   
Q 74+9c         T 83   ☑ 83  
Q 979+4c        T 983  ☑ 983 
Q 867+1         T 868  ☑ 868 
Q 423+6         T 429  ☑ 429 
Q 11-8z         T 3    ☑ 3   
Q 508+50        T 558  ☑ 558 

-------------------

36000/36000 [==============================] - 17s 480us/step - loss: 0.0032 - acc: 0.9992 - val_loss: 0.0288 - val_acc: 0.9915
Q 632+98b       T 730  ☑ 730 
Q 7+7c          T 14   ☑ 14  
Q 5-4           T 1    ☑ 1   
Q 30-4z         T 26   ☑ 26  
Q 867-703       T 164  ☒ 174 
Q 388+4c        T 392  ☑ 392 
Q 6+1           T 7    ☑ 7   
Q 5+4           T 9    ☑ 9   
Q 7+6c          T 13   ☑ 13  
Q 129+80        T 209  ☑ 209 

--------------------------------------------------
Iteration 53
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 18s 487us/step - loss: 0.0011 - acc: 0.9998 - val_loss: 0.0252 - val_acc: 0.9926
Q 88-6          T 82   ☑ 82  
Q 391-0         T 391  ☑ 391 
Q 8-0           T 8    ☑ 8   
Q 9+9c          T 18   ☑ 18  
Q 6-1           T 5    ☑ 5   
Q 85-8z         T 77   ☑ 77  
Q 162-2         T 160  ☑ 160 
Q 68+3c         T 71   ☑ 71  
Q 465+3         T 468  ☑ 468 
Q 7-5           T 2    ☑ 2   

-------------------

36000/36000 [==============================] - 17s 479us/step - loss: 0.0027 - acc: 0.9993 - val_loss: 0.0293 - val_acc: 0.9918
Q 856-30        T 826  ☑ 826 
Q 157-55        T 102  ☑ 102 
Q 503-3         T 500  ☑ 500 
Q 9+4c          T 13   ☑ 13  
Q 81+33b        T 114  ☑ 114 
Q 637-64y       T 573  ☑ 573 
Q 258+58c       T 316  ☑ 316 
Q 558+60b       T 618  ☑ 618 
Q 834-30        T 804  ☑ 804 
Q 447+65c       T 512  ☑ 512 

--------------------------------------------------
Iteration 66
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 18s 492us/step - loss: 0.0024 - acc: 0.9993 - val_loss: 0.0346 - val_acc: 0.9900
Q 9+7c          T 16   ☑ 16  
Q 78+18c        T 96   ☑ 96  
Q 95+8c         T 103  ☑ 103 
Q 558+60b       T 618  ☑ 618 
Q 100-1z        T 99   ☑ 99  
Q 622+94b       T 716  ☑ 716 
Q 596-4         T 592  ☑ 592 
Q 9-3           T 6    ☑ 6   
Q 137-84y       T 53   ☑ 53  
Q 63-3          T 60   ☑ 60  

-------------------

36000/36000 [==============================] - 16s 437us/step - loss: 0.0032 - acc: 0.9991 - val_loss: 0.0301 - val_acc: 0.9918
Q 705-3         T 702  ☑ 702 
Q 7-5           T 2    ☑ 2   
Q 961-241       T 720  ☑ 720 
Q 843+72b       T 915  ☑ 915 
Q 928-853y      T 75   ☑ 75  
Q 376+26c       T 402  ☑ 402 
Q 512-379zy     T 133  ☑ 133 
Q 4-4           T 0    ☑ 0   
Q 13+5          T 18   ☑ 18  
Q 834-30        T 804  ☑ 804 

--------------------------------------------------
Iteration 79
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 16s 435us/step - loss: 0.0025 - acc: 0.9993 - val_loss: 0.0277 - val_acc: 0.9927
Q 68-9z         T 59   ☑ 59  
Q 3-0           T 3    ☑ 3   
Q 70-50         T 20   ☑ 20  
Q 599+64cb      T 663  ☑ 663 
Q 6-3           T 3    ☑ 3   
Q 456-3         T 453  ☑ 453 
Q 9+4c          T 13   ☑ 13  
Q 455+4         T 459  ☑ 459 
Q 504+472       T 976  ☑ 976 
Q 13+6          T 19   ☑ 19  

-------------------

36000/36000 [==============================] - 16s 435us/step - loss: 0.0016 - acc: 0.9995 - val_loss: 0.0279 - val_acc: 0.9929
Q 423+6         T 429  ☑ 429 
Q 134-6z        T 128  ☑ 128 
Q 934-127z      T 807  ☑ 807 
Q 507-1         T 506  ☑ 506 
Q 60-5z         T 55   ☑ 55  
Q 901-799zy     T 102  ☒ 112 
Q 653+301       T 954  ☒ 964 
Q 223+14        T 237  ☑ 237 
Q 49-7          T 42   ☑ 42  
Q 552+6         T 558  ☑ 558 

--------------------------------------------------
Iteration 92
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 16s 436us/step - loss: 6.3168e-04 - acc: 0.9999 - val_loss: 0.0227 - val_acc: 0.9939
Q 3+2           T 5    ☑ 5   
Q 403+3         T 406  ☑ 406 
Q 76+1          T 77   ☑ 77  
Q 211-28zy      T 183  ☑ 183 
Q 681-7z        T 674  ☑ 674 
Q 178+2         T 180  ☑ 180 
Q 48-4          T 44   ☑ 44  
Q 438-53y       T 385  ☑ 385 
Q 3-2           T 1    ☑ 1   
Q 16+1          T 17   ☑ 17  

---------------

## Save Model

In [11]:
# model.save('BiLSTM.h5') 

## Validation

In [42]:
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
#     print('Q', q[::-1] if REVERSE else q, end=' ')
#     print('T', correct, end=' ')
    if correct == guess:
#         print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
#     else:
#         print(colors.fail + '☒' + colors.close, end=' ')
#     print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))


MSG : Accuracy is 0.9748
